<a href="https://colab.research.google.com/github/YapingWu/GoogleColab/blob/main/genpass/genpass_from_github.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# github代码复制到google colab

参考资料：
1. https://techsupportallbugs.wordpress.com/2018/06/05/using-git-with-colab-via-ssh/

## 配置SSH
1. 上传压缩后的密钥对文件和配置文件
1. 将密钥对文件和配置文件解压到/root/.ssh文件夹中
2. 将私钥加载到本地ssh-agent中
3. 设置git账户

In [4]:
!rm -rf ~/.ssh
!unzip '/content/ssh-colab.zip' -d ~/.ssh
!chmod 700 ~/.ssh

# add the ssh server as a hnown host
!touch ~/.ssh/known_hosts
!ssh-keyscan github.com >> ~/.ssh/known_hosts
!chmod 644 ~/.ssh/known_hosts

Archive:  /content/ssh-colab.zip
  inflating: /root/.ssh/config       
  inflating: /root/.ssh/id_rsa_colab  
  inflating: /root/.ssh/id_rsa_colab.pub  
# github.com:22 SSH-2.0-babeld-383743ad
# github.com:22 SSH-2.0-babeld-c2015ddc
# github.com:22 SSH-2.0-babeld-383743ad


In [5]:
!ssh-agent /bin/bash 

# 以下命令在ssh-agent启动的shell中执行
# chmod 600 ~/.ssh/id_rsa_colab  # 私钥需要设置仅自己可以访问，才能添加到代理
# ssh-add ~/.ssh/id_rsa_colab
# ssh-add -l

bash: cannot set terminal process group (57): Inappropriate ioctl for device
bash: no job control in this shell
/content/drive/My Drive/gen_pass# chmod 600 ~/.ssh/id_rsa_colab
/content/drive/My Drive/gen_pass# ssh-add ~/.ssh/id_rsa_colab
Identity added: /root/.ssh/id_rsa_colab (/root/.ssh/id_rsa_colab)
/content/drive/My Drive/gen_pass# EXIT
bash: EXIT: command not found
/content/drive/My Drive/gen_pass# exit
exit


In [6]:
!git config --global user.name 'colab'
!git config --global user.email 'vyapings@163.com'

## clone私有库
1. 安装google云盘
2. 将工作目录更改为云端硬盘内的文件夹
3. 运行git clone。

In [1]:
from google.colab import drive
drive.mount('./drive')

import os
os.chdir('/content/drive/MyDrive')

!pwd
!ls

Mounted at ./drive
/content/drive/MyDrive
'Anyfile Notepad Files'  'Colab Notebooks'   gen_pass


In [9]:
# !rm -rf gen_pass
# !git clone git@github.com:YapingWu/gen_pass.git

import os
os.chdir('/content/drive/MyDrive/gen_pass')
# !git stash
!git pull origin master

remote: Enumerating objects: 11, done.
remote: Counting objects: 100% (11/11), done.
remote: Compressing objects: 100% (1/1), done.
remote: Total 6 (delta 5), reused 6 (delta 5), pack-reused 0
Unpacking objects: 100% (6/6), done.
From github.com:YapingWu/gen_pass
 * branch            master     -> FETCH_HEAD
   99d5e3e..6dceed7  master     -> origin/master
Updating 99d5e3e..6dceed7
Fast-forward
 main.py                     |  4 +++-
 src/utils/evaluate_model.py | 46 ++++++++++++++++++++++++++-------------------
 2 files changed, 30 insertions(+), 20 deletions(-)


# 运行genpass项目

上传并解压原始密码数据集

In [ ]:
!unzip '/content/dataset.zip' -d '/content/drive/MyDrive/gen_pass/dataset'

Archive:  /content/dataset.zip
  inflating: /content/drive/MyDrive/gen_pass/dataset/phpbb.txt  
  inflating: /content/drive/MyDrive/gen_pass/dataset/rockyou.txt  
  inflating: /content/drive/MyDrive/gen_pass/dataset/myspace.txt  


In [2]:
import os
print(os.getcwd())
os.chdir('/content/drive/MyDrive/gen_pass')
print(os.getcwd())

/content
/content/drive/MyDrive/gen_pass


## 数据分析

In [7]:
!python main.py \
  --stats-only \
  --infile dataset/phpbb.txt dataset/myspace.txt

2021-03-25 04:40:35.212460: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.11.0
2021-03-25 04:40:38 [INFO] 
##########################################################################################################
# GENPass
# 1 PL模型预测。预测密码中的下一个单元（unit）。
# 2 权重选择。按照权重选择从3个PL模型的结果中选择权重最高的unit，并根据wordlist随机选择一个密码进行替换。
# 3 分类器。使用CNN模型预测上一步生成的密码属于3个数据集的概率。
# 4 判定器。选择出在3个数据集概率相差不大的密码作为结果输出。
#
##########################################################################################################

2021-03-25 04:40:39 [INFO] Start...
2021-03-25 04:40:39 [INFO] dataset/phpbb.txt数据集中的密码数量：184379
2021-03-25 04:40:39 [INFO] 密码长度不在[4, 20]区间的密码个数：1013，占总密码数量的百分比：0.55%
2021-03-25 04:40:39 [INFO] dataset/myspace.txt数据集中的密码数量：37144
2021-03-25 04:40:39 [INFO] 密码长度不在[4, 20]区间的密码个数：259，占总密码数量的百分比：0.70%
2021-03-25 04:40:39 [INFO] 分析结果图保存到文件：result/figure/password.png
2021-03-25 04:40:40 [INFO] End...


## 数据预处理

In [5]:
!python main.py \
  --pre-processing-only \
  --infile dataset/phpbb.txt dataset/myspace.txt

2021-03-25 13:16:17.712334: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.11.0
2021-03-25 13:16:19 [INFO] 
##########################################################################################################
# GENPass
# 1 PL模型预测。预测密码中的下一个单元（unit）。
# 2 权重选择。按照权重选择从3个PL模型的结果中选择权重最高的unit，并根据wordlist随机选择一个密码进行替换。
# 3 分类器。使用CNN模型预测上一步生成的密码属于3个数据集的概率。
# 4 判定器。选择出在3个数据集概率相差不大的密码作为结果输出。
#
##########################################################################################################

2021-03-25 13:16:19 [INFO] Start...
2021-03-25 13:16:19 [INFO] 原始密码集划分为测试集和训练集
2021-03-25 13:16:20 [INFO] dataset/phpbb.txt：train shape：(147503, 1), test shape：(36876, 1)
2021-03-25 13:16:21 [INFO] dataset/myspace.txt：train shape：(29715, 1), test shape：(7429, 1)
2021-03-25 13:16:21 [INFO] 开始预处理
2021-03-25 13:16:21 [INFO] 待处理的文件名称：dataset/phpbb.txt
2021-03-25 13:16:21 [INFO] PCFG编码
2021-03-25 13:16:22 [INFO] 去重后的编码序列个数：2873， 保存到文件：r

## 训练word-level LSTM

### 训练分词器

In [ ]:
!python main.py \
    --train --model tokenizer \
    --dnames myspace phpbb myspace_part phpbb_part

2021-03-24 12:30:28.043468: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.11.0
2021-03-24 12:30:29 [INFO] 
##########################################################################################################
# GENPass
# 1 PL模型预测。预测密码中的下一个单元（unit）。
# 2 权重选择。按照权重选择从3个PL模型的结果中选择权重最高的unit，并根据wordlist随机选择一个密码进行替换。
# 3 分类器。使用CNN模型预测上一步生成的密码属于3个数据集的概率。
# 4 判定器。选择出在3个数据集概率相差不大的密码作为结果输出。
#
##########################################################################################################

2021-03-24 12:30:29 [INFO] Start...
2021-03-24 12:30:29 [INFO] 加载编码后的密码数据，数据文件：result/preprocessed/wordlist/myspace.txt
2021-03-24 12:30:29 [INFO] 开始训练tokenizer模型
2021-03-24 12:30:29 [INFO] 将tokenizer模型保存到文件中：result/model/tokenizer/myspace.pkl
2021-03-24 12:30:29 [INFO] 加载编码后的密码数据，数据文件：result/preprocessed/wordlist/phpbb.txt
2021-03-24 12:30:29 [INFO] 开始训练tokenizer模型
2021-03-24 12:30:29 [INFO] 将tokenizer模型保存到文件中：result/model/tokeniz

### 使用pcfg编码结果训练LSTM模型

In [ ]:
!python main.py \
    -e pro \
    --train \
    --model lstm \
    --dnames myspace phpbb myspace_part phpbb_part

2021-03-24 12:33:42.034927: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.11.0
2021-03-24 12:33:43 [INFO] 
##########################################################################################################
# GENPass
# 1 PL模型预测。预测密码中的下一个单元（unit）。
# 2 权重选择。按照权重选择从3个PL模型的结果中选择权重最高的unit，并根据wordlist随机选择一个密码进行替换。
# 3 分类器。使用CNN模型预测上一步生成的密码属于3个数据集的概率。
# 4 判定器。选择出在3个数据集概率相差不大的密码作为结果输出。
#
##########################################################################################################

2021-03-24 12:33:43 [INFO] Start...
2021-03-24 12:33:43 [INFO] 开始加载编码后的密码数据：result/preprocessed/wordlist/myspace.txt
2021-03-24 12:33:43 [INFO] 开始加载tokenizer模型：result/model/tokenizer/myspace.pkl
2021-03-24 12:33:43 [INFO] 将编码后的密码转换为（整数）序列
2021-03-24 12:33:43 [INFO] Total Sequences: 7586
2021-03-24 12:33:43 [INFO] 创建LSTM模型的输入输出
2021-03-24 12:33:43 [INFO] X Shape: (7586, 19), y Shape: (7586,)
2021-03-24 12:33:43 [INFO] 划分训练集、验证集和测试集


## 训练char-level CNN

In [ ]:
!python main.py \
    -e pro \
    --train \
    --model cnn \
    --dnames myspace phpbb

2021-03-24 13:22:18.775391: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.11.0
2021-03-24 13:22:20 [INFO] 
##########################################################################################################
# GENPass
# 1 PL模型预测。预测密码中的下一个单元（unit）。
# 2 权重选择。按照权重选择从3个PL模型的结果中选择权重最高的unit，并根据wordlist随机选择一个密码进行替换。
# 3 分类器。使用CNN模型预测上一步生成的密码属于3个数据集的概率。
# 4 判定器。选择出在3个数据集概率相差不大的密码作为结果输出。
#
##########################################################################################################

2021-03-24 13:22:20 [INFO] Start...
2021-03-24 13:22:20 [INFO] 开始加载数据...
2021-03-24 13:22:20 [INFO] dataset/myspace.txt 数据大小：37144
2021-03-24 13:22:20 [INFO] dataset/phpbb.txt 数据大小：184379
2021-03-24 13:22:20 [INFO] 训练模型的总数据量：77144
2021-03-24 13:22:20 [INFO] 划分验证集和测试集
2021-03-24 13:22:21 [INFO] x_val Shape: (15428, 100), y_val Shape: (15428, 2)
2021-03-24 13:22:21 [INFO] x_test Shape: (15428, 100), y_test Shape: (15428, 2)
2021-03-2

## 生成密码

In [ ]:
!python main.py \
  --generate genpass \
  -c 0.2 \
  --alphas 6 \
  --chunk 100000

In [ ]:
!python main.py \
  --generate pl \
  --dnames myspace phpbb myspace_part phpbb_part\
  --alphas 6 \
  --chunk 100000

2021-03-25 07:44:25.651283: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.11.0
2021-03-25 07:44:27 [INFO] 
##########################################################################################################
# GENPass
# 1 PL模型预测。预测密码中的下一个单元（unit）。
# 2 权重选择。按照权重选择从3个PL模型的结果中选择权重最高的unit，并根据wordlist随机选择一个密码进行替换。
# 3 分类器。使用CNN模型预测上一步生成的密码属于3个数据集的概率。
# 4 判定器。选择出在3个数据集概率相差不大的密码作为结果输出。
#
##########################################################################################################

2021-03-25 07:44:27 [INFO] Start...
2021-03-25 07:44:27 [INFO] 使用 myspace PL模型生成密码
2021-03-25 07:44:27 [INFO] 创建PL模型。数据集名称：myspace
2021-03-25 07:44:27.733464: I tensorflow/compiler/jit/xla_cpu_device.cc:41] Not creating XLA devices, tf_xla_enable_xla_devices not set
2021-03-25 07:44:27.734616: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcuda.so.1
2021-03-25 07:44:27.746518:

## 评估模型

In [10]:
!python main.py --evaluate --alphas 3 4 5 6

2021-03-25 14:11:44.819287: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.11.0
2021-03-25 14:11:46 [INFO] 
##########################################################################################################
# GENPass
# 1 PL模型预测。预测密码中的下一个单元（unit）。
# 2 权重选择。按照权重选择从3个PL模型的结果中选择权重最高的unit，并根据wordlist随机选择一个密码进行替换。
# 3 分类器。使用CNN模型预测上一步生成的密码属于3个数据集的概率。
# 4 判定器。选择出在3个数据集概率相差不大的密码作为结果输出。
#
##########################################################################################################

2021-03-25 14:11:46 [INFO] Start...
2021-03-25 14:11:46 [INFO] PL模型评估。评估方法：cross-site
2021-03-25 14:11:47 [INFO] 测试集文件：result/preprocessed/rockyou_small.txt，密码个数：99970
2021-03-25 14:11:47 [INFO] 测试集文件：result/preprocessed/rockyou_small.txt，密码个数：99970
2021-03-25 14:11:47 [INFO] data_name：myspace, alpha：3, unique value：999, test_size: 7000, guessed: 1, mr：0.01%
2021-03-25 14:11:47 [INFO] data_name：myspace, alpha：4, unique value：9922, 